In [6]:
!pip install demoji emoji bs4 Levenshtein spacy_langdetect wordsegment xlrd openpyxl

     |████████████████████████████████| 133kB 28.4MB/s 
     |████████████████████████████████| 163kB 46.5MB/s 
     |████████████████████████████████| 4.8MB 47.3MB/s 
     |████████████████████████████████| 1.0MB 52.7MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.7-cp37-none-any.whl size=993460 sha256=6bef9d03625c4b6bb4d1af6d129478f619e7217c3b7745ac08e1e2cba1afb089
  Stored in directory: /root/.cache/pip/wheels/ec/0c/a9/1647275e7ef5014e7b83ff30105180e332867d65e7617ddafe
Successfully built langdetect


In [7]:
#Basic Python and Machine learning libraries
import os, sys, warnings, random, time, re, math, string, demoji, emoji, copy
import pandas as pd
import numpy as np
import Levenshtein
import matplotlib.pyplot as plt
import networkx as nx

from string import punctuation
from collections import Counter
from re import search
import seaborn as sns
from scipy import stats
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold

from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS as sklearn_stop_words
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel

from bs4 import BeautifulSoup
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk.tokenize.casual import casual_tokenize
from nltk.util import ngrams

import spacy
from spacy_langdetect import LanguageDetector

warnings.filterwarnings('ignore')
demoji.download_codes()

#tqdm with pandas
from tqdm import tqdm
from scipy import stats
from tqdm import tqdm_notebook
tqdm.pandas()
from collections import Counter

# sklearn data science models
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression, SGDClassifier, Lasso
from sklearn.svm import LinearSVC
import xgboost as xgb

from wordsegment import load, segment

... OK (Got response in 0.14 seconds)
Writing emoji data to /root/.demoji/codes.json ...
... OK


In [8]:
def basic_clean(text):
    # text = emoji.demojize(text)
    text = str(text)
    text = demoji.replace_with_desc(text, sep = " ") #Emoji to text
    text = BeautifulSoup(text, 'lxml').get_text() #links
    text = re.sub('https?://\S+|www\.\S+', '', text) #links
    text = re.sub(' +', ' ', text) 
    text = re.sub("^RT", "", text) #remove RT
    text = re.sub(' +', ' ', text) 
    text = re.sub("^QT", "", text) #remove QT
    text = re.sub(' +', ' ', text) 
    text = re.sub('\n', " ", text) #newlines
    text = re.sub(' +', ' ', text) 
    text = str(text).lower() #makes everything lowercase
    emoji_pattern = re.compile("["
                          "\U0001F1E0-\U0001F1FF"  # flags (iOS)
                          "\U0001F300-\U0001F5FF"  # symbols & pictographs
                          "\U0001F600-\U0001F64F"  # emoticons
                          "\U0001F680-\U0001F6FF"  # transport & map symbols
                          "\U0001F700-\U0001F77F"  # alchemical symbols
                          "\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                          "\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                          "\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                          "\U0001FA00-\U0001FA6F"  # Chess Symbols
                          "\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                          "\U00002702-\U000027B0"  # Dingbats
                          "\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r' ', text)
    text = re.sub(' +', ' ', text) 
    text = text.replace('@', ' ').replace('#',' ').replace('_',' ') #removes @ # _ from mentions and user-handles, does not remove the handle itself
    return text
  
def camel_case_split(identifier):
  matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', identifier)
  return " ".join([m.group(0) for m in matches])

def clean_all(text):
    text = basic_clean(text)
    text = camel_case_split(text)

    return text

In [11]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer('english')
lemmatizer = WordNetLemmatizer()

contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", 
                       "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", 
                       "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", 
                       "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am",
                       "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", 
                       "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have",
                       "it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not",
                       "mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", 
                       "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                       "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", 
                       "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have",
                       "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is",
                       "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would",
                       "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have",
                       "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have",
                       "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", 
                       "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did",
                       "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", 
                       "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", 
                       "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                       "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have",
                       "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", 'u.s':'america', 'e.g':'for example'}

punct = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-",
                 "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 
                 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', '!':' '}

mispell_dict = {'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater',
                'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ',
                'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can',
                'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 
                'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 
                'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 
                'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization',
                'demonetisation': 'demonetization'}

def clean_text(text):
    '''Clean emoji, Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = emoji.demojize(text)
    text = re.sub(r'\:(.*?)\:','',text)
    text = str(text).lower()    #Making Text Lowercase
    text = re.sub('\[.*?\]', '', text)
    #The next 2 lines remove html text
    text = BeautifulSoup(text, 'lxml').get_text()
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",", "'")
    text = re.sub(r"[^a-zA-Z?.!,¿']+", " ", text)
    return text

def clean_contractions(text, mapping):
    '''Clean contraction using contraction mapping'''    
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    for word in mapping.keys():
        if ""+word+"" in text:
            text = text.replace(""+word+"", ""+mapping[word]+"")
    #Remove Punctuations
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    text = re.sub(r"([?.!,¿])", r" \1 ", text)
    text = re.sub(r'[" "]+', " ", text)
    return text

def clean_special_chars(text, punct, mapping):
    '''Cleans special characters present(if any)'''   
    for p in mapping:
        text = text.replace(p, mapping[p])
    
    for p in punct:
        text = text.replace(p, f' {p} ')
    
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}  
    for s in specials:
        text = text.replace(s, specials[s])
    
    return text

def correct_spelling(x, dic):
    '''Corrects common spelling errors'''   
    for word in dic.keys():
        x = x.replace(word, dic[word])
    return x

def remove_space(text):
    '''Removes awkward spaces'''   
    #Removes awkward spaces 
    text = text.strip()
    text = text.split()
    return " ".join(text)

def clean_gm(tweet): 
            
    # Special characters
    tweet = re.sub(r"\x89Û_", "", tweet)
    tweet = re.sub(r"\x89ÛÒ", "", tweet)
    tweet = re.sub(r"\x89ÛÓ", "", tweet)
    tweet = re.sub(r"\x89ÛÏWhen", "When", tweet)
    tweet = re.sub(r"\x89ÛÏ", "", tweet)
    tweet = re.sub(r"let\x89Ûªs", "let's", tweet)
    tweet = re.sub(r"\x89Û÷", "", tweet)
    tweet = re.sub(r"\x89Ûª", "", tweet)
    tweet = re.sub(r"\x89Û\x9d", "", tweet)
    tweet = re.sub(r"å_", "", tweet)
    tweet = re.sub(r"\x89Û¢", "", tweet)
    tweet = re.sub(r"\x89Û¢åÊ", "", tweet)
    tweet = re.sub(r"åÊ", "", tweet)
    tweet = re.sub(r"åÈ", "", tweet)
    tweet = re.sub(r"Ì©", "e", tweet)
    tweet = re.sub(r"å¨", "", tweet)
    tweet = re.sub(r"åÇ", "", tweet)
    tweet = re.sub(r"åÀ", "", tweet)
    
    return tweet

def text_preprocessing_pipeline(text):
    '''Cleaning and parsing the text.'''
    text = clean_gm(text)
    text = clean_text(text)
    text = clean_contractions(text, contraction_mapping)
    text = clean_special_chars(text, punct, punct_mapping)
    text = correct_spelling(text, mispell_dict)
    text = remove_space(text)
    return text

def nlp_preprocessing(text, remove_stopwords=True, stemming=False, lemmatization=True):
    '''Stopword removal + Stemming + Lemmatization'''
    word_tokens = word_tokenize(text)
    if remove_stopwords:
        word_tokens = [w for w in word_tokens if not w in stop_words]
    if stemming: #Random Cutting, try avoiding it
        word_tokens = [stemmer.stem(w) for w in word_tokens]
    if lemmatization:
        word_tokens = [lemmatizer.lemmatize(w) for w in word_tokens]
    return " ".join(word_tokens)

def preprocess_english(pandas_series):
    pandas_series = pandas_series.progress_apply(lambda x:text_preprocessing_pipeline(x))
    pandas_series = pandas_series.progress_apply(lambda x:nlp_preprocessing(x))

    return pandas_series


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


# New section

In [12]:
import torch
from torchtext.legacy import data
from torch import nn, optim
from torch.optim import lr_scheduler

In [13]:
class config:
  TRAIN_ARTICLES = 'Data/labeled_articles.csv'
  VAL_ARTICLES = 'Data/labeled_articles.csv'
  TEST_ARTICLES = 'Data/labeled_articles.csv'
  TRAIN_TWEETS = 'Data/labeled_tweet.csv'
  VAL_TWEETS = 'Data/labeled_tweet.csv'
  TEST_TWEETS =  'Data/labeled_tweet.csv'
  SEED = 713
  LOSS = nn.BCEWithLogitsLoss()
  LR_SCHEDULER = None
  DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  N_WORKERS = 4
  MAX_VOCAB_SIZE = 25_000
  BATCH_SIZE = 64
  EMBEDDINGS = "glove.6B.100d"
  EMBEDDING_DIM = 100
  HIDDEN_DIM = 256
  OUTPUT_DIM = 1
  N_LAYERS = 2
  BIDIRECTIONAL = True
  DROPOUT = 0.5
  N_EPOCHS = 70   

In [14]:
from torchtext.legacy import data

class DataFrameDataset(data.Dataset):

    def __init__(self, df, text_field, label_field, is_test=False, **kwargs):
        fields = [('Text', text_field), ('label', label_field)]
        examples = []
        for i, row in df.iterrows():
            label = row.label if not is_test else None
            text = row.Text
            examples.append(data.Example.fromlist([text, label], fields))

        super().__init__(examples, fields, **kwargs)

    @staticmethod
    def sort_key(ex):
        return len(ex.Text)

    @classmethod
    def splits(cls, text_field, label_field, train_df, val_df=None, test_df=None, **kwargs):
        train_data, val_data, test_data = (None, None, None)

        if train_df is not None:
            train_data = cls(train_df.copy(), text_field, label_field, **kwargs)
        if val_df is not None:
            val_data = cls(val_df.copy(), text_field, label_field, **kwargs)
        if test_df is not None:
            test_data = cls(test_df.copy(), text_field, label_field, True, **kwargs)

        return tuple(d for d in (train_data, val_data, test_data) if d is not None)

In [15]:
import torch
from torch import nn

class RNN_TWEET(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        #text = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(text))
        
        #embedded = [sent len, batch size, emb dim]
        
        #pack sequence
        # lengths need to be on CPU!
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.to('cpu'))
        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        
        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
                
        #hidden = [batch size, hid dim * num directions]
            
        return self.fc(hidden)


In [16]:
import warnings, random, time
import pandas as pd
import numpy as np
import torch
from sklearn import metrics

warnings.filterwarnings('ignore')

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

def data_lao():        
    train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
        (train_ds, val_ds, test_ds), 
        batch_size = BATCH_SIZE,
        sort_within_batch = True,
        device = device)

def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        text, text_lengths = batch.text
        
        predictions = model(text, text_lengths).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

# if __name__ == '__main__':
# #     tokenizer = BertTokenizer.from_pretrained('bert-large-uncased', do_lower_case=True)
# #     model = BertForSequenceClassification.from_pretrained(
# #     'bert-large-uncased', # Use the 124-layer, 1024-hidden, 16-heads, 340M parameters BERT model with an uncased vocab.
# #     num_labels = 2, # The number of output labels--2 for binary classification. You can increase this for multi-class tasks.   
# #     output_attentions = False, # Whether the model returns attentions weights.
# #     output_hidden_states = False, # Whether the model returns all hidden-states.
# # )


In [17]:
import pandas as pd
import numpy as np
# import ipdb
import spacy 
import torch, time
from tqdm import tqdm
from torch import nn, optim
from torchtext.legacy import data


tqdm.pandas()

def run():
    training_df = pd.read_csv('Data/labeled_tweet.csv')
    # ipdb.set_trace()
    training_df['text'] = training_df['text'].apply(clean_all)
    validation_df = pd.read_csv('Data/labeled_tweet.csv')
    validation_df['text'] = validation_df['text'].apply(clean_all)

    TEXT = data.Field(tokenize = 'spacy',
                    tokenizer_language = 'en_core_web_sm',
                    include_lengths = True)
    LABEL = data.LabelField(dtype = torch.float)

    train_ds, val_ds, test_ds = DataFrameDataset.splits(
        text_field=TEXT, label_field=LABEL, train_df=training_df, val_df=validation_df, test_df=validation_df)
    # ipdb.set_trace()    
    TEXT.build_vocab(train_ds, max_size = config.MAX_VOCAB_SIZE, vectors = config.EMBEDDINGS, 
                 unk_init = torch.Tensor.normal_)
    LABEL.build_vocab(train_ds)
    
    train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
        (train_ds, val_ds, test_ds), 
        batch_size = config.BATCH_SIZE,
        sort_within_batch = True,
        device = config.DEVICE)

    pretrained_embeddings = TEXT.vocab.vectors
    INPUT_DIM = len(TEXT.vocab)
    PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
    UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

    model = RNN_TWEET(INPUT_DIM, 
                config.EMBEDDING_DIM, 
                config.HIDDEN_DIM, 
                config.OUTPUT_DIM, 
                config.N_LAYERS, 
                config.BIDIRECTIONAL, 
                config.DROPOUT, 
                PAD_IDX)
    optimizer = optim.Adam(model.parameters())
    criterion = config.LOSS
    criterion = criterion.to(config.DEVICE)

    model.embedding.weight.data.copy_(pretrained_embeddings)
    model.embedding.weight.data[UNK_IDX] = torch.zeros(config.EMBEDDING_DIM)
    model.embedding.weight.data[PAD_IDX] = torch.zeros(config.EMBEDDING_DIM)

    model = model.to(config.DEVICE)

    best_valid_loss = float('inf')

    for epoch in range(config.N_EPOCHS):

        start_time = time.time()
        
        train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
        valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
        
        end_time = time.time()

        epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), 'Models/rnn_tweet.pt')
        
        print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
        print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

In [19]:
df = pd.read_csv('/content/drive/MyDrive/Submission/Train_Data/evaluation_data_corrected.csv')
df.head()

,Text_ID,Text
0,article_4001,Market Highlights\n\nAn automotive constant ve...
1,article_4002,Overview:\n\nBe it as a means for food or as p...
2,article_4003,"GenScript USA Inc. (U.S.), Horizon Discovery G..."
3,article_4004,New Delhi: Prime Minister Narendra Modi spoke ...
4,article_4005,"New Delhi, Feb 1 (PTI) The Delhi High Court Mo..."


In [20]:
df.Text = df.Text.apply(clean_all)

In [21]:
df_article, df_tweet = df.iloc[:222, :], df.iloc[222:, :]

In [22]:
def predict_sentiment(model, sentence):
    nlp = spacy.load('en_core_web_sm')
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    device = 'cuda'
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return np.round(prediction.item())

In [23]:
import pickle
def save_vocab(vocab, path):
    output = open(path, 'wb')
    pickle.dump(vocab, output)
    output.close()
def load_vocab(path):
    output = open(path, 'rb')
    vocab = pickle.load(output)
    return vocab

In [26]:
TEXT = data.Field(tokenize = 'spacy',
                tokenizer_language = 'en_core_web_sm',
                include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)
TEXT.vocab = load_vocab('/content/drive/MyDrive/Submission/Classification/Models/article_text_vocab.pkl')
LABEL.vocab = load_vocab('/content/drive/MyDrive/Submission/Classification/Models/article_label_vocab.pkl')
pretrained_embeddings = TEXT.vocab.vectors
INPUT_DIM = len(TEXT.vocab)
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
model_article = RNN_TWEET(INPUT_DIM, 
                config.EMBEDDING_DIM, 
                config.HIDDEN_DIM, 
                config.OUTPUT_DIM, 
                config.N_LAYERS, 
                config.BIDIRECTIONAL, 
                config.DROPOUT, 
                PAD_IDX)
model_article.load_state_dict(torch.load('/content/drive/MyDrive/Submission/Classification/Models/rnn_article.pt'))


<All keys matched successfully>

In [27]:
device = 'cuda'
model_article.to(device)

RNN_TWEET(
  (embedding): Embedding(17685, 100, padding_idx=1)
  (rnn): LSTM(100, 256, num_layers=2, dropout=0.5, bidirectional=True)
  (fc): Linear(in_features=512, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [28]:
TEXT = data.Field(tokenize = 'spacy',
                tokenizer_language = 'en_core_web_sm',
                include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)
TEXT.vocab = load_vocab('/content/drive/MyDrive/Submission/Classification/Models/tweet_text_vocab.pkl')
LABEL.vocab = load_vocab('/content/drive/MyDrive/Submission/Classification/Models/tweet_label_vocab.pkl')
pretrained_embeddings = TEXT.vocab.vectors
INPUT_DIM = len(TEXT.vocab)
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
model_tweet = RNN_TWEET(INPUT_DIM, 
                config.EMBEDDING_DIM, 
                config.HIDDEN_DIM, 
                config.OUTPUT_DIM, 
                config.N_LAYERS, 
                config.BIDIRECTIONAL, 
                config.DROPOUT, 
                PAD_IDX)
model_tweet.load_state_dict(torch.load('/content/drive/MyDrive/Submission/Classification/Models/rnn_tweet.pt'))
model_tweet.to(device)

RNN_TWEET(
  (embedding): Embedding(3045, 100, padding_idx=1)
  (rnn): LSTM(100, 256, num_layers=2, dropout=0.5, bidirectional=True)
  (fc): Linear(in_features=512, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [36]:
def append_sentiment_labels(df, model):
  df['Mobile_Tech'] = np.zeros(len(df))
  for i in range(len(df)):
    df.Mobile_Tech.iloc[i] = predict_sentiment(model, df.Text.iloc[i])


In [37]:
%%time
append_sentiment_labels(df_article, model_article)
append_sentiment_labels(df_tweet, model_tweet)

CPU times: user 3min 37s, sys: 3.14 s, total: 3min 40s
Wall time: 3min 40s


In [38]:
df_article

,Text_ID,Text,Mobile_Tech
0,article_4001,market highlights an automotive constant veloc...,0.0
1,article_4002,overview: be it as a means for food or as pets...,0.0
2,article_4003,"genscript usa inc. (u.s.), horizon discovery g...",0.0
3,article_4004,new delhi: prime minister narendra modi spoke ...,1.0
4,article_4005,"new delhi, feb 1 (pti) the delhi high court mo...",1.0
...,...,...,...
217,article_4220,motorola released an update to fix one of the ...,0.0
218,article_4221,the company has rolled out software update to ...,0.0
219,article_4222,some xiaomi and redmi devices are facing a pec...,0.0
220,article_4223,"mi, redmi, and poco users are facing an issue ...",0.0


In [39]:
df_tweet

,Text_ID,Text,Mobile_Tech
222,tweet_4001,lloydsbank how do i report a scam on my accou...,0.0
223,tweet_4002,iameraijaz: i have been denied security in j...,0.0
224,tweet_4003,fplprop gooner fpl i think there's a risk sa...,0.0
225,tweet_4004,brfootball: official: bayern munich loan def...,0.0
226,tweet_4005,"timinhonolulu: 5. so far, three 2016 cycle u...",0.0
...,...,...,...
427,tweet_4206,ios 6.1 for iphone very bad and iphone5 ...,1.0
428,tweet_4207,one bad thing of xiaomiredminote4 is the aut...,1.0
429,tweet_4208,i have a concern over the service provided . p...,1.0
430,tweet_4209,bought nokia6.1plus in oct2018 and having wor...,1.0


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [40]:
company = ['10.or',
 'acer',
 'adcom',
 'airtel',
 'alcatel',
 'alpha',
 'amazon',
 'aoc',
 'apple',
 'aqua',
 'archos',
 'asus',
 'benq',
 'billion',
 'black shark',
 'blackberry',
 'blu',
 'bq',
 'byond',
 'celkon',
 'centric',
 'champion computers',
 'champone',
 'cherry mobile',
 'comio',
 'coolpad',
 'creo',
 'croma',
 'datawind',
 'detel',
 'docoss',
 'doogee',
 'elari',
 'elephone',
 'energizer',
 'essential',
 'evercoss',
 'fairphone',
 'geeksphone',
 'general mobile',
 'gionee',
 'google',
 'homtom',
 'honor',
 'hp',
 'htc',
 'huawei',
 'huawai',
 'hyve',
 'i-mobiles',
 'iball',
 'iberry',
 'infinix',
 'infocus',
 'intex',
 'itel',
 'ivoomi',
 'jio',
 'jivi',
 'jolla',
 'josh mobile',
 'karbonn',
 'kestrel',
 'kodak',
 'kult',
 'kyocera',
 'land rover',
 'lava',
 'lemon',
 'lenovo',
 'lg',
 'lumigon',
 'm-tech',
 'mafe mobile',
 'magicon',
 'marshall',
 'maxx mobile',
 'meizu',
 'mercury',
 'micromax',
 'microsoft',
 'mitashi',
 'mito',
 'mobiistar',
 'motorola',
 'mphone',
 'mts',
 'myphone',
 'namotel',
 'neffos',
 'nexian',
 'nextbit',
 'nokia',
 'nubia',
 'nuu mobile',
 'obi worldphone',
 'oneplus',
 'onida',
 'oplus',
 'oppo',
 'oukitel',
 'panasonic',
 'pantel',
 'pepsi',
 'phicomm',
 'philips',
 'poco',
 'polaroid',
 'porsche design',
 'qiku',
 'qmobile',
 'razer',
 'reach',
 'realme',
 'reliance',
 'ringing bells',
 'rio',
 'royole',
 'salora',
 'samsung',
 'sansui',
 'saygus',
 'sharp',
 'sikur',
 'silent circle',
 'simmtronics',
 'sirin',
 'smartisan',
 'smartron',
 'sony',
 'spice',
 'sunstrike',
 'swipe',
 'symphony',
 't mobile',
 'tambo',
 'tcl',
 'tecno',
 'tonino lamborghini',
 'unihertz',
 'vaio',
 'vertu',
 'viaan',
 'videocon',
 'vivo',
 'vodafone',
 'wickedleak',
 'wiio',
 'xiaomi',
 'xolo',
 'yamada denki',
 'yandex',
 'yota devices',
 'yu',
 'zen',
 'ziox',
 'zopo',
 'zte',
 'zuk',
 'zync']

In [41]:
map_sentiments = {2:'positive',1:'neutral',0:'negative'}

In [46]:
pwd

'/content/drive/MyDrive/Submission'

In [44]:
%cd /content/drive/MyDrive/Submission

/content/drive/MyDrive/Submission


In [45]:
from ASGCN.infer import Inferer


Infer = Inferer()
allOutput = pd.DataFrame()

ModuleNotFoundError: ignored

In [ ]:
pat = re.compile("|".join(map(re.escape, company))

In [ ]:
for i in range(len(df_tweet)):
  text = df_tweet.Text.iloc[i]
  predicted_brands = list(set(pat.findall(text)))
  temp_df = pd.DataFrame()
  mobile_tech = df_tweet['Mobile_Tech'][i]
  if mobile_tech==1:
    for j in range(len(predicted_brands)):
      brand = predicted_brands[j]
      ABSA_model_out = Infer.evaluate(text,brand)
      sentiment = map_sentiments[ABSA_model_out]
      temp_df1 = pd.DataFrame({'Text_ID': df_tweet['Text_ID'][i],
                            'Mobile_Tech': mobile_tech,
                            'Brands_Entity_Identified':brand, 
                            'Sentiment_Identified':sentiment,}, index = [0])    
      temp_df = temp_df.append(temp_df1)
  else:
    brand = ""
    sentiment = ""
    temp_df1 = pd.DataFrame({'Text_ID': df_tweet['Text_ID'][i],
                            'Mobile_Tech': mobile_tech,
                            'Brands_Entity_Identified':brand, 
                            'Sentiment_Identified':sentiment,}, index = [0])    
    temp_df = temp_df.append(temp_df1)
  allOutput = allOutput.append(temp_df)

In [ ]:
for i in range(len(df_article)):
  text = df_article.Text.iloc[i]
  predicted_brands = list(set(pat.findall(text)))
  temp_df = pd.DataFrame()
  if mobile_tech==1:
    for j in range(len(predicted_brands)):
      brand = predicted_brands[j]
      ABSA_model_out = Infer.evaluate(text,brand)
      sentiment = map_sentiments[ABSA_model_out]
      temp_df1 = pd.DataFrame({'Text_ID': df_article['Text_ID'][i],
                            'Mobile_Tech': mobile_tech,
                            'Brands_Entity_Identified':brand, 
                            'Sentiment_Identified':sentiment,}, index = [0])    
      temp_df = temp_df.append(temp_df1)
  else:
    brand = ""
    sentiment = ""
    temp_df1 = pd.DataFrame({'Text_ID': df_article['Text_ID'][i],
                            'Mobile_Tech': mobile_tech,
                            'Brands_Entity_Identified':brand, 
                            'Sentiment_Identified':sentiment,}, index = [0]) 
  allOutput = allOutput.append(temp_df)

In [ ]:
allOutput.to_csv("output2.csv")

In [ ]:
from heading_generation import generate_headlines
import copy

In [ ]:
labels = df_article["Mobile_Tech"]==1

In [ ]:
df_heading  = copy.copy(df_article[labels])

In [ ]:
headlines = generate_headlines(dataframe = df_headling, PATH = './models/news.pth')

In [ ]:
df_heading["Heading"] = headlines

In [ ]:
df_article["Heading"] = [""]*len(df_article)
df_tweet["Heading"] = [""]*len(df_tweet)

In [ ]:
df_article = df_article.drop(~labels, axis=0)
df_article.append(df_heading)
df_final = pd.DataFrame()
df_final.append(df_tweet)
df_final.append(df_article)

In [ ]:
df_final.to_csv("output1.csv")